In [1]:
strip_number_digits = True

# Raw Data Preparation

In [2]:
import os
import pickle as pk

path_goldstandard = './daten_goldstandard'

# Restore metadata so far
with open(os.path.join(path_goldstandard, 'columns_metadata.pkl'), 'rb') as handle:
    columns_metadata_dict = pk.load(handle)

In [3]:
for k in columns_metadata_dict.keys():
    print(k, '\n', columns_metadata_dict[k], '\n')

data_analysis_columns 
 ['coordinate_E', 'coordinate_N', 'corporate_full', 'doi', 'edition', 'exactDate', 'format_prefix', 'format_postfix', 'isbn', 'ismn', 'musicid', 'part', 'person_100', 'person_700', 'person_245c', 'pubinit', 'scale', 'ttlfull_245', 'ttlfull_246', 'volumes'] 

columns_to_use 
 ['duplicates', 'coordinate_E_x', 'coordinate_E_y', 'coordinate_N_x', 'coordinate_N_y', 'corporate_full_x', 'corporate_full_y', 'doi_x', 'doi_y', 'edition_x', 'edition_y', 'exactDate_x', 'exactDate_y', 'format_prefix_x', 'format_prefix_y', 'format_postfix_x', 'format_postfix_y', 'isbn_x', 'isbn_y', 'ismn_x', 'ismn_y', 'musicid_x', 'musicid_y', 'part_x', 'part_y', 'person_100_x', 'person_100_y', 'person_700_x', 'person_700_y', 'person_245c_x', 'person_245c_y', 'pubinit_x', 'pubinit_y', 'scale_x', 'scale_y', 'ttlfull_245_x', 'ttlfull_245_y', 'ttlfull_246_x', 'ttlfull_246_y', 'volumes_x', 'volumes_y'] 

similarity_metrics 
 {'coordinate_E': LCSStr({'qval': 1, 'external': True}), 'coordinate_N': L

In [4]:
import json

records = []
file = 'slave.json'

for line in open(os.path.join(path_goldstandard, file), 'r'):
    records.append(json.loads(line))

print('Number of records in data file {:s}\t{:d}'.format(file, len(records)))

Number of records in data file slave.json	615


Die eingelesenen Rohdaten als DataFrame.

In [5]:
import pandas as pd

rd = pd.DataFrame(records)

# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(rd.columns)

rd.head()

,docid,035liste,isbn,ttlfull,ttlpart,person,corporate,pubyear,decade,century,exactDate,edition,part,pages,volumes,pubinit,pubword,scale,coordinate,doi,ismn,musicid,format
0,00350560X,"[(OCoLC)884555343, (ABN)000305947]",[],{'245': ['Sideways']},{'245': ['Sideways']},"{'100': [], '700': ['GiamattiPaul', 'Haden Chu...","{'110': [], '710': [], '810': []}",2005,2005,2005,2005,,[],[1 DVD-Video (ca. 122 Min.)],[1 DVD-Video (ca. 122 Min.)],[],[],,[],[],[],2785408,[VM010300]
1,003894924,"[(OCoLC)314603155, (ABN)000414274]",[],{'245': ['Li regres Nostre Dame']},{'245': ['Li regres Nostre Dame']},"{'100': ['Huon Le Roi'], '700': ['LångforsArth...","{'110': [], '710': [], '810': []}",1907,1907,1907,1907,,[],"[CXLVII, 210 p.]","[CXLVII, 210 p.]",[],[],,[],[],[],,[BK020300]
2,004308069,"[(OCoLC)887407580, (ABN)000363324]",[],{'245': ['Euricius Cordus Epigrammata (1520)']},{'245': ['Euricius Cordus Epigrammata (1520)']},"{'100': ['CordusEuricius'], '700': ['KrauseKar...","{'110': [], '710': [], '810': []}",1892,1892,1892,1892,,[Heft 5],[1 Band],[1 Band],[],[],,[],[],[],,[BK020000]
3,005021073,"[(OCoLC)841563357, (ABN)000288211, R004141449]",[],{'245': ['Je voudrais que quelqu'un m'attende ...,{'245': ['Je voudrais que quelqu'un m'attende ...,"{'100': ['GavaldaAnna'], '700': [], '800': [],...","{'110': [], '710': [], '810': []}",2006,2006,2006,2006,,[],[3 Compact Discs],[3 Compact Discs],[],[],,[],[],[],A,[MU030100]
4,005179726,"[(OCoLC)611172738, (ABN)000232404]","[3-7281-1755-2, 3-519-05031-5]","{'245': ['Städtebau in der Schweiz 1800-1990',...","{'245': ['Städtebau in der Schweiz 1800-1990',...","{'100': ['KochMichael'], '700': [], '800': [],...","{'110': [], '710': [], '810': []}",1992,1992,1992,1992,,[Nr. 81],[315 S.],[315 S.],[],[],,[],[],[],,[BK020000]


In [6]:
print('Total expected number of pairs of data to be deduplicated : {:,d}'.format(len(rd)**2))

Total expected number of pairs of data to be deduplicated : 378,225


## Transform Attributes for Similarity Comparison

In [7]:
import data_preparation_funcs as dpf

rd = dpf.attribute_preprocessing(rd, columns_metadata_dict['data_analysis_columns'], strip_number_digits)

### ismn

In [8]:
display(rd.ismn[rd.ismn.apply(lambda x : len(x)>0)])

17        m001068048
173       m001068062
205       m001068048
269       m004182772
334       m001068048
349    9790004182772
Name: ismn, dtype: object

## Build Pairs

In [9]:
# Helper column for joining in full outer
rd['joiner'] = 1
rd_pairs = pd.merge(left=rd, right=rd, on='joiner')
# Drop helper column again
rd_pairs.drop(columns='joiner', inplace=True)
rd_pairs.reset_index(drop=True, inplace=True)

print('Number of paired records to be deduplicated {:,d}'.format(len(rd_pairs)))

# Extend display to number of columns of DataFrame
pd.options.display.max_columns = len(rd_pairs)

rd_pairs.head()

Number of paired records to be deduplicated 378,225


,docid_x,035liste_x,isbn_x,ttlpart_x,pubyear_x,decade_x,century_x,exactDate_x,edition_x,part_x,pages_x,volumes_x,pubinit_x,pubword_x,scale_x,coordinate_x,doi_x,ismn_x,musicid_x,coordinate_E_x,coordinate_N_x,corporate_110_x,corporate_710_x,corporate_full_x,format_prefix_x,format_postfix_x,person_100_x,person_700_x,person_245c_x,ttlfull_245_x,ttlfull_246_x,docid_y,035liste_y,isbn_y,ttlpart_y,pubyear_y,decade_y,century_y,exactDate_y,edition_y,part_y,pages_y,volumes_y,pubinit_y,pubword_y,scale_y,coordinate_y,doi_y,ismn_y,musicid_y,coordinate_E_y,coordinate_N_y,corporate_110_y,corporate_710_y,corporate_full_y,format_prefix_y,format_postfix_y,person_100_y,person_700_y,person_245c_y,ttlfull_245_y,ttlfull_246_y
0,00350560X,"[(OCoLC)884555343, (ABN)000305947]",[],{'245': ['Sideways']},2005,2005,2005,2005uuuu,,,[1 DVD-Video (ca. 122 Min.)],1 122,,[],,[],,,2785408,,,,,,vm,010300,,"giamattipaul, haden churchthomas, madsenvirgin...",regie: alexader payne ; drehbuch: alexander pa...,sideways,,00350560X,"[(OCoLC)884555343, (ABN)000305947]",[],{'245': ['Sideways']},2005,2005,2005,2005uuuu,,,[1 DVD-Video (ca. 122 Min.)],1 122,,[],,[],,,2785408,,,,,,vm,010300,,"giamattipaul, haden churchthomas, madsenvirgin...",regie: alexader payne ; drehbuch: alexander pa...,sideways,
1,00350560X,"[(OCoLC)884555343, (ABN)000305947]",[],{'245': ['Sideways']},2005,2005,2005,2005uuuu,,,[1 DVD-Video (ca. 122 Min.)],1 122,,[],,[],,,2785408,,,,,,vm,010300,,"giamattipaul, haden churchthomas, madsenvirgin...",regie: alexader payne ; drehbuch: alexander pa...,sideways,,003894924,"[(OCoLC)314603155, (ABN)000414274]",[],{'245': ['Li regres Nostre Dame']},1907,1907,1907,1907uuuu,,,"[CXLVII, 210 p.]",210,,[],,[],,,,,,,,,bk,020300,huon le roi,långforsarthur,par huon le roi de cambrai; publ. d'aprs tous ...,li regres nostre dame,
2,00350560X,"[(OCoLC)884555343, (ABN)000305947]",[],{'245': ['Sideways']},2005,2005,2005,2005uuuu,,,[1 DVD-Video (ca. 122 Min.)],1 122,,[],,[],,,2785408,,,,,,vm,010300,,"giamattipaul, haden churchthomas, madsenvirgin...",regie: alexader payne ; drehbuch: alexander pa...,sideways,,004308069,"[(OCoLC)887407580, (ABN)000363324]",[],{'245': ['Euricius Cordus Epigrammata (1520)']},1892,1892,1892,1892uuuu,,5,[1 Band],1,,[],,[],,,,,,,,,bk,020000,corduseuricius,krausekarl,euricius cordus; herausgegeben von karl krause,euricius cordus epigrammata (1520),
3,00350560X,"[(OCoLC)884555343, (ABN)000305947]",[],{'245': ['Sideways']},2005,2005,2005,2005uuuu,,,[1 DVD-Video (ca. 122 Min.)],1 122,,[],,[],,,2785408,,,,,,vm,010300,,"giamattipaul, haden churchthomas, madsenvirgin...",regie: alexader payne ; drehbuch: alexander pa...,sideways,,005021073,"[(OCoLC)841563357, (ABN)000288211, R004141449]",[],{'245': ['Je voudrais que quelqu'un m'attende ...,2006,2006,2006,2006uuuu,,,[3 Compact Discs],3,,[],,[],,,,,,,,,mu,030100,gavaldaanna,,anna gavalda,je voudrais que quelqu'un m'attende quelque part,
4,00350560X,"[(OCoLC)884555343, (ABN)000305947]",[],{'245': ['Sideways']},2005,2005,2005,2005uuuu,,,[1 DVD-Video (ca. 122 Min.)],1 122,,[],,[],,,2785408,,,,,,vm,010300,,"giamattipaul, haden churchthomas, madsenvirgin...",regie: alexader payne ; drehbuch: alexander pa...,sideways,,005179726,"[(OCoLC)611172738, (ABN)000232404]","[3-7281-1755-2, 3-519-05031-5]","{'245': ['Städtebau in der Schweiz 1800-1990',...",1992,1992,1992,1992uuuu,,81,[315 S.],315,,[],,[],,,,,,,,,bk,020000,kochmichael,,michael koch,"städtebau in der schweiz 1800-1990, entwicklun...",


### DataFrame Handover

In [10]:
import bz2

# Binary intermediary metadata file
with open(os.path.join(path_goldstandard,
                       'rd_columns_metadata.pkl'), 'wb') as dict_output_file:
    pk.dump(columns_metadata_dict, dict_output_file)

# Store into compressed intermediary file
with bz2.BZ2File(os.path.join(path_goldstandard, 'rd_feature_base_df.pkl'),
                 'w') as df_output_file:
    pk.dump(rd_pairs, df_output_file)